Alunos: Daniel de Paula, Gustavo Guerreiro e Mayara Cardoso Simões

# Trabalho Final de Aprendizado de Máquina sobre Visão Computacional: Classificação de Imagens de Cães e Gatos

O dataset utilizado é de propriedade da Microsoft e está disponível em: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset

## Separação dos Dados em Treino, Teste e Validação

A primeira etapa da implementação é a de separação dos dados. Inicialmente o diretório se encontra no formato:
```
PetImages/
├── Cat
└── Dog
```

Como uma Rede Neural necessita de uma separação entre treino, teste e possivelmente validação, o dataset será reorganizado para seguir a seguinte estrutura mais comum nesse tipo de implementação:
```
dataset/
├── train/
│   ├── Cat/
│   └── Dog/
├── val/
│   ├── Cat/
│   └── Dog/
└── test/
    ├── Cat/
    └── Dog/
```
Para usar essa estrutura se utilizou a classe GeneratorBasedBuilder do TensorFlow para fazer a divisão mais eficiente e monstar a estrutura em treino, validação e teste.

## Importação das bibliotecas

In [162]:
from os import path
from glob import glob
from random import Random
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow_datasets.core import GeneratorBasedBuilder, DatasetInfo, Version

import imghdr
import tensorflow as tf
import tensorflow_datasets as tfds

## Definição da Classe do Dataset

Para organizar o dataset de uma forma mais eficiente foi criada a classe CatsDogs.
Inicialmente se tem uma função auxiliar chamada listar_imagens_validas, ela é usada para checar se a imagem sendo tratada é de fato um jpg válido ou foi corrompido.

Já a classe CatsDogs em si possui três métodos:
* _info: contém as informações contidas no dataset, no caso uma imagem de 3 dimensões (altura, largura e cor RGB) e o rótulo podendo ser "Cat" ou Dog.
* _split_generators: método principal que busca nas pastas as imagens dos gatos e cães, separa cada grupo em treino, validação e teste e então junta as imagens de cada animal.
* _generate_examples: usado para gerar os dados retornados em si, pegando cada imagem e atribuindo um id para ela.

In [163]:
def listar_imagens_validas(pasta, label):
    caminhos = glob(f"{pasta}/*")
    validos = []
    for caminho in caminhos:
        if imghdr.what(caminho) == "jpeg":
            validos.append((caminho, label))
    return validos


class CatsDogs(GeneratorBasedBuilder):
    VERSION = Version("1.0.0")
    SEED = 42
    PASTA_PADRAO = "PetImages"

    def _info(self):
        return DatasetInfo(
            builder=self,
            features=tfds.features.FeaturesDict({
                "image": tfds.features.Image(shape=(None, None, 3)),
                "label": tfds.features.ClassLabel(names=["Cat", "Dog"]),
            })
        )

    def _split_generators(self, dl_manager):
        raiz = self.PASTA_PADRAO

        gatos = listar_imagens_validas(f"{raiz}/Cat", 0)
        caes = listar_imagens_validas(f"{raiz}/Dog", 1)

        gatos_treino, gatos_resto = train_test_split(gatos, test_size=0.3, random_state=self.SEED)
        gatos_val, gatos_test = train_test_split(gatos_resto, test_size=0.5, random_state=self.SEED)

        caes_treino, caes_resto = train_test_split(caes, test_size=0.3, random_state=self.SEED)
        caes_val, caes_test = train_test_split(caes_resto, test_size=0.5, random_state=self.SEED)

        rng = Random(self.SEED)
        treino = gatos_treino + caes_treino
        rng.shuffle(treino)

        val = gatos_val + caes_val
        rng.shuffle(val)

        teste = gatos_test + caes_test
        rng.shuffle(teste)

        return {
            "train": self._generate_examples(treino),
            "val": self._generate_examples(val),
            "test": self._generate_examples(teste)
        }

    def _generate_examples(self, arquivos):
        for caminho, rotulo in arquivos:
            if not path.isfile(caminho):
                continue
            yield caminho, {"image": caminho, "label": rotulo}


## Instanciamento dos Datasets
A classe CatsDogs é instanciada e os datasets são construídos e carredos em variáveis.

In [165]:
builder = CatsDogs()
builder.download_and_prepare()

ds_train = builder.as_dataset(split="train")
ds_val = builder.as_dataset(split="val")
ds_test = builder.as_dataset(split="test")

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 2337 examples [00:01, 2335.83 examples/s]
Generating train examples...: 5175 examples [00:02, 2630.61 examples/s]
Generating train examples...: 8043 examples [00:03, 2738.71 examples/s]
Generating train examples...: 11017 examples [00:04, 2831.22 examples/s]
Generating train examples...: 13944 examples [00:05, 2865.41 examples/s]
Generating train examples...: 16944 examples [00:06, 2911.04 examples/s]
                                                                        
Shuffling C:\Users\guto_\tensorflow_datasets\cats_dogs\incomplete.28UBWP_1.0.0\cats_dogs-train.tfrecord*...:   0%|          | 0/17320 [00:00<?, ? examples/s]
Generating splits...:  33%|███▎      | 1/3 [00:06<00:13,  6.93s/ splits]                                                                                     
Generating val examples...: 0 examples [00:00, ?

Dataset cats_dogs downloaded and prepared to C:\Users\guto_\tensorflow_datasets\cats_dogs\1.0.0. Subsequent calls will reuse this data.


## Pré-Processamento
As imagens são pré processados, tendo inicialmente o seu tamanho ajustado e então os seus valores normalizados do formato RGB (0-255, 0-255, 0-255) para (0.0-1.0, 0.0-1.0, 0.0-1.0).

In [164]:
TAMANHO = 128

def preprocessamento(dicionario):
    image = dicionario["image"]
    label = dicionario["label"]
    image = tf.image.resize(image, (TAMANHO, TAMANHO))
    image = tf.cast(image, tf.float32) / 255.0

    return image, label

Além de aplicar o pré-processamento, os dados são organizados para serem divididos em batches para facilitar o processamento na rede e o prefetch para agilizar o processo de carregamento dos batches enquanto a rede é treinada.

In [166]:
BATCH = 32
ds_train = ds_train.map(preprocessamento).batch(BATCH).prefetch(tf.data.AUTOTUNE)
ds_val   = ds_val.map(preprocessamento).batch(BATCH).prefetch(tf.data.AUTOTUNE)
ds_test  = ds_test.map(preprocessamento).batch(BATCH).prefetch(tf.data.AUTOTUNE)

## Construção do Modelo
O modelo possui uma arquitetura sequencial, as etapas são as seguintes:
* Recebe um input de tamanho (128, 128, 3)
* Uma camada Convolucional com 32 filtros de formato 3x3.
* Max Pooling de janelas 2x2.
* Uma camada Convolucional com 64 filtros de formato 3x3.
* Max Pooling de janelas 2x2.
* Camadas são achatadas.
* Camada densa inicial com 64 pontos de entrada.
* Camada final de saída com 2 valores de ativação possíveis (cão ou gato).

In [167]:
def construir_modelo():
    modelo = models.Sequential([
        layers.Input(shape=(TAMANHO, TAMANHO, 3)),

        layers.Conv2D(32, (3, 3,), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])

    modelo.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return modelo

## Treinamento
O modelo arquitetado é então rodado.

In [171]:

model = construir_modelo()
historico = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5
)


Epoch 1/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 35s 63ms/step - accuracy: 0.6785 - loss: 0.5922 - val_accuracy: 0.7435 - val_loss: 0.5078
Epoch 2/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 33s 61ms/step - accuracy: 0.7869 - loss: 0.4546 - val_accuracy: 0.7737 - val_loss: 0.4599
Epoch 3/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 34s 62ms/step - accuracy: 0.8312 - loss: 0.3761 - val_accuracy: 0.7788 - val_loss: 0.4833
Epoch 4/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 35s 65ms/step - accuracy: 0.8706 - loss: 0.3021 - val_accuracy: 0.7880 - val_loss: 0.5015
Epoch 5/5
542/542 ━━━━━━━━━━━━━━━━━━━━ 36s 66ms/step - accuracy: 0.8977 - loss: 0.2386 - val_accuracy: 0.7786 - val_loss: 0.5741
